In [15]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
import numpy as np
from pathlib import Path

team_number = 20
url1 = "https://fantasy.premierleague.com/api/bootstrap-static/"
url2 = "https://fantasy.premierleague.com/api/entry/698498/history/"
url3 = "https://fantasy.premierleague.com/api/event/6/live/"
url4 = "https://fantasy.premierleague.com/api/fixtures"

#NaNs to zeros
def toint(a):
    if np.isnan(a):
        return 0
    else: return int(a)

#If no matches played not to devide by zero
def noZ(a):
    b = a.copy()
    for i in range(len(b)):
        if b[i] == 0:
            b[i]=1
    return b


p1 = requests.get(url1)
page1 = BeautifulSoup(p1.text)
data1 = str(page1.p)[3:-4]
d1 = json.loads(data1)
bigTable = pd.DataFrame(d1['elements'])
bigTable = bigTable[['team', 'element_type', 'web_name', 'goals_scored', 'assists', 'bonus', 'event_points', 'total_points', 
               'saves', 'own_goals', 'clean_sheets', 'penalties_missed', 'penalties_saved', 'yellow_cards', 'red_cards', 
               'minutes', 'bps', 'creativity', 'threat', 'ict_index', 'influence',
               'value_season', 'form', 'value_form', 'points_per_game', 
               'goals_conceded', 
               'in_dreamteam', 'dreamteam_count',
               'now_cost', 'cost_change_event', 'cost_change_event_fall',
               'cost_change_start', 'cost_change_start_fall', 'selected_by_percent',
               'transfers_in_event', 'transfers_out_event', 'transfers_in', 'transfers_out', 
               'chance_of_playing_this_round', 'chance_of_playing_next_round', 'news_added', 'news', 'status', 
               'ep_this', 'ep_next', 'first_name', 'second_name', 'team_code', 'id', 'photo', 'special', 'squad_number', 'code']]
bigTable['full_name'] = bigTable['first_name'] + ' ' + bigTable['second_name']
bigTable.to_csv(Path('in/fpltable.csv'))

p4 = requests.get(url4)
page4 = BeautifulSoup(p4.text)
data4 = str(page4.p)[3:-4]
d4 = json.loads(data4)
Fixtures = pd.DataFrame(d4)
Fixtures.to_csv(Path('in/fplfixtures.csv'))

#Figuring out ended lastGameweek   lastGW

if Fixtures.at[1,'minutes'] != 90:
    lastGW = 0
elif Fixtures.at[len(Fixtures)-1,'minutes'] == 90:
    lastGW = Fixtures.at[len(Fixtures)-1,'event']
else:
    for i in range(1,len(Fixtures)):
        if Fixtures.at[i,'minutes'] != 90:
            lastGW = Fixtures.at[i-1,'event']
            break
lastGW = int(lastGW)

Gameweeks = pd.DataFrame()
for i in range(1,2*team_number - 1):
    url = "https://fantasy.premierleague.com/api/event/" + str(i) + "/live/"
    p = requests.get(url)
    page = BeautifulSoup(p.text)
    data = str(page.p)[3:-4]
    d = json.loads(data)
    nexTour = pd.DataFrame(d['elements'])
    
    if not nexTour.empty:
        nt1 = pd.DataFrame(nexTour['stats'].tolist())
        nt1['id'] = nexTour['id']
        nt1['gameweek'] = i
        nt1.index = nt1['gameweek']*1000+nt1['id']
        Gameweeks = Gameweeks.append(nt1)
        print(i)


teams = dict(zip(pd.DataFrame(d1['teams'])['id'],pd.DataFrame(d1['teams'])['name']))
players = dict(zip(bigTable['id'],bigTable['full_name']))
teamplayers = dict(zip(bigTable['id'],bigTable['team']))

Gameweeks['team'] = [teamplayers[i] for i in Gameweeks['id']]

Gameweeks['threat'] = pd.to_numeric(Gameweeks['threat'])
Gameweeks['creativity'] = pd.to_numeric(Gameweeks['creativity'])

Gameweeks['team_a'] = [int(Fixtures[(Fixtures['event'] == Gameweeks.iloc[i,20]) & \
                                         ((Fixtures['team_a'] == Gameweeks.iloc[i,21])|\
                                          (Fixtures['team_h'] == Gameweeks.iloc[i,21]))]['team_a']) \
                            for i in range(len(Gameweeks))]
Gameweeks['team_h'] = [int(Fixtures[(Fixtures['event'] == Gameweeks.iloc[i,20]) & \
                                         ((Fixtures['team_a'] == Gameweeks.iloc[i,21])|\
                                          (Fixtures['team_h'] == Gameweeks.iloc[i,21]))]['team_h']) \
                            for i in range(len(Gameweeks))]

Gameweeks['teamAgainst'] = [Gameweeks.at[i,'team_a'] if Gameweeks.at[i,'team'] == Gameweeks.at[i,'team_h']\
                            else Gameweeks.at[i,'team_h'] \
                           for i in Gameweeks.index]
Gameweeks['side'] = ['home' if Gameweeks.at[i,'team'] == Gameweeks.at[i,'team_h']\
                            else 'away' \
                           for i in Gameweeks.index]

del Gameweeks['team_a']
del Gameweeks['team_h']


Gameweeks.to_csv(Path('in/fplgameweeks.csv'))
Gameweeks

1
2
3
4
5
6
7
8
9
10
11
12
13
14


,assists,bonus,bps,clean_sheets,creativity,goals_conceded,goals_scored,ict_index,in_dreamteam,influence,...,red_cards,saves,threat,total_points,yellow_cards,id,gameweek,team,teamAgainst,side
1014,0,1,27,1,0.0,0,0,1.8,False,18.0,...,0,2,0.0,7,0,14,1,1,13,away
1027,0,0,0,0,0.0,0,0,0.0,False,0.0,...,0,0,0.0,0,0,27,1,2,17,away
1047,0,2,29,1,0.0,0,0,2.5,False,25.4,...,0,3,0.0,9,0,47,1,4,18,away
1048,0,0,0,0,0.0,0,0,0.0,False,0.0,...,0,0,0.0,0,0,48,1,4,18,away
1072,0,0,0,0,0.0,0,0,0.0,False,0.0,...,0,0,0.0,0,0,72,1,3,15,home
1073,0,0,0,0,0.0,0,0,0.0,False,0.0,...,0,0,0.0,0,0,73,1,3,15,home
1074,0,0,0,0,0.0,0,0,0.0,False,0.0,...,0,0,0.0,0,0,74,1,3,15,home
1093,0,1,28,1,10.0,0,0,3.6,False,25.6,...,0,3,0.0,8,0,93,1,5,16,home
1094,0,0,15,0,0.0,3,0,2.4,False,24.2,...,0,3,0.0,2,0,94,1,2,17,away
1095,0,0,0,0,0.0,0,0,0.0,False,0.0,...,0,0,0.0,0,0,95,1,5,16,home


In [16]:
#Team tables

print(0)
#1. Creating  a table with average threat and GW threats for teams

TeamThreat = pd.DataFrame()
TeamThreat['id'] = pd.DataFrame(d1['teams'])['id']
TeamThreat['Team'] = pd.DataFrame(d1['teams'])['name']
TeamThreat['Threat av'] = np.zeros(len(TeamThreat))
TeamThreat['Matches'] = np.zeros(len(TeamThreat))

for j in range(lastGW,0,-1):
    TeamThreat['Threat GW'+str(j)] = [Gameweeks[(Gameweeks['team']==i)&(Gameweeks['gameweek']==j)]['threat'].sum() \
                                      for i in range(1,team_number+1)]
    TeamThreat['Matches'] = TeamThreat['Matches'] + (TeamThreat['Threat GW'+str(j)] != np.zeros(len(TeamThreat)))
    

TeamThreat['Threat av'] = [Gameweeks[Gameweeks['team']==i]['threat'].sum() for i in range(1,team_number+1)] \
    /noZ(TeamThreat['Matches'])
print(1)
#2. Creating  a table with average creativity and GW creativities for teams

TeamCreativity = pd.DataFrame()
TeamCreativity['id'] = pd.DataFrame(d1['teams'])['id']
TeamCreativity['Team'] = pd.DataFrame(d1['teams'])['name']
TeamCreativity['Creativity av'] = np.zeros(len(TeamCreativity))
TeamCreativity['Matches'] = np.zeros(len(TeamCreativity))

for j in range(lastGW,0,-1):
    TeamCreativity['Creativity GW'+str(j)] = [Gameweeks[(Gameweeks['team']==i)&(Gameweeks['gameweek']==j)] \
            ['creativity'].sum() for i in range(1,team_number+1)]
    TeamCreativity['Matches'] = TeamCreativity['Matches'] + \
            (TeamCreativity['Creativity GW'+str(j)] != np.zeros(len(TeamCreativity)))
    

TeamCreativity['Creativity av'] = [Gameweeks[Gameweeks['team']==i]['creativity'].sum() for i in range(1,team_number+1)] \
        /noZ(TeamCreativity['Matches'])
print(2)
#3. Creating  a table with average threat allowed by teams and GW threat allowed

TableDefence = pd.DataFrame()
TableDefence['id'] = pd.DataFrame(d1['teams'])['id']
TableDefence['Team'] = pd.DataFrame(d1['teams'])['name']
TableDefence['Threat allowed av'] = np.zeros(len(TableDefence))
TableDefence['Matches'] = np.zeros(len(TableDefence))

for j in range(lastGW,0,-1):
    TableDefence['Threat allowed GW'+str(j)] = [Gameweeks[(Gameweeks['teamAgainst']==i)&(Gameweeks['gameweek']==j)] \
            ['threat'].sum() for i in range(1,team_number+1)]
    TableDefence['Matches'] = TableDefence['Matches'] + \
            (TableDefence['Threat allowed GW'+str(j)] != np.zeros(len(TableDefence)))

TableDefence['Threat allowed av'] = [Gameweeks[Gameweeks['teamAgainst']==i]['threat'].sum() for i in range(1,team_number+1)] \
        /noZ(TableDefence['Matches'])

threatAllowedAv = TableDefence['Threat allowed av'].mean()
print(3)
#4. Creating  a table with average adjusted threat and GW threats adj for teams

TeamThreatAd = pd.DataFrame()
TeamThreatAd['id'] = pd.DataFrame(d1['teams'])['id']
TeamThreatAd['Team'] = pd.DataFrame(d1['teams'])['name']
TeamThreatAd['Threat av adj'] = np.zeros(len(TeamThreatAd))
TeamThreatAd['Matches'] = np.zeros(len(TeamThreatAd))

for j in range(lastGW,0,-1):    
    TeamThreatAd['Threat GW'+str(j)+' adj'] = [TeamThreat.at[i-1,'Threat GW'+str(j)]*threatAllowedAv/ \
                TableDefence.at[int(Gameweeks[(Gameweeks['team']==i)&(Gameweeks['gameweek']==j)]['teamAgainst'].mean()-1), \
                'Threat allowed av'] \
                                             for i in range(1,team_number+1)]
    TeamThreatAd['Matches'] = TeamThreatAd['Matches'] + (TeamThreatAd['Threat GW'+str(j)+' adj'] != np.zeros(len(TeamThreat)))
    TeamThreatAd['Threat av adj'] = TeamThreatAd['Threat av adj']  + TeamThreatAd['Threat GW'+str(j)+' adj']

TeamThreatAd['Threat av adj'] = TeamThreatAd['Threat av adj']/noZ(TeamThreatAd['Matches'])
print(4)
#5. Creating  a table with average adjusted creativity and GW creativities adj for teams

TeamCreativityAd = pd.DataFrame()
TeamCreativityAd['id'] = pd.DataFrame(d1['teams'])['id']
TeamCreativityAd['Team'] = pd.DataFrame(d1['teams'])['name']
TeamCreativityAd['Creativity av adj'] = np.zeros(len(TeamCreativityAd))
TeamCreativityAd['Matches'] = np.zeros(len(TeamCreativityAd))

for j in range(lastGW,0,-1):    
    TeamCreativityAd['Creativity GW'+str(j)+' adj'] = [TeamCreativity.at[i-1,'Creativity GW'+str(j)]*threatAllowedAv/ \
                TableDefence.at[int(Gameweeks[(Gameweeks['team']==i)&(Gameweeks['gameweek']==j)]['teamAgainst'].mean()-1), \
                'Threat allowed av'] \
                                             for i in range(1,team_number+1)]
    TeamCreativityAd['Matches'] = TeamCreativityAd['Matches'] + (TeamCreativityAd['Creativity GW'+str(j)+' adj'] != \
                                                                 np.zeros(len(TeamCreativity)))
    TeamCreativityAd['Creativity av adj'] = TeamCreativityAd['Creativity av adj']  + \
        TeamCreativityAd['Creativity GW'+str(j)+' adj']

TeamCreativityAd['Creativity av adj'] = TeamCreativityAd['Creativity av adj']/noZ(TeamThreatAd['Matches'])
print(5)
#6. Creating  a table with average threat allowed adjusted by teams and GW threat allowed adjusted

TableDefenceAd = pd.DataFrame()
TableDefenceAd['id'] = pd.DataFrame(d1['teams'])['id']
TableDefenceAd['Team'] = pd.DataFrame(d1['teams'])['name']
TableDefenceAd['Threat allowed av adj'] = np.zeros(len(TableDefenceAd))
TableDefenceAd['Matches'] = np.zeros(len(TableDefenceAd))

for j in range(lastGW,0,-1):    
    TableDefenceAd['Threat allowed GW'+str(j)+' adj'] = [TableDefence.at[i-1,'Threat allowed GW'+str(j)]*threatAllowedAv/ \
                TeamThreat.at[int(Gameweeks[(Gameweeks['team']==i)&(Gameweeks['gameweek']==j)]['teamAgainst'].mean()-1), \
                'Threat av'] \
                                             for i in range(1,team_number+1)]
    TableDefenceAd['Matches'] = TableDefenceAd['Matches'] + (TableDefenceAd['Threat allowed GW'+str(j)+' adj'] != \
                                                                 np.zeros(len(TableDefenceAd)))
    TableDefenceAd['Threat allowed av adj'] = TableDefenceAd['Threat allowed av adj']  + \
        TableDefenceAd['Threat allowed GW'+str(j)+' adj']

TableDefenceAd['Threat allowed av adj'] = TableDefenceAd['Threat allowed av adj']/noZ(TableDefenceAd['Matches'])

print(6)


#Total Team Table

TableTeams = pd.DataFrame()
TableTeams['id'] = pd.DataFrame(d1['teams'])['id']
TableTeams['Team'] = pd.DataFrame(d1['teams'])['name']

TableTeams['Threat adjusted'] = TeamThreatAd['Threat av adj']
TableTeams['Threat'] = TeamThreat['Threat av']
TableTeams['Creativity adjusted'] = TeamCreativityAd['Creativity av adj']
TableTeams['Creativity'] = TeamCreativity['Creativity av']
TableTeams['Threat allowed adjusted'] = TableDefenceAd['Threat allowed av adj']
TableTeams['Threat allowed'] = TableDefence['Threat allowed av']
print(7)



#PLayer Tables

#PlayerMatches = pd.DataFrame(list(players.keys()), columns = ['id'])
PlayerMatches = pd.DataFrame()
PlayerMatches['id'] = bigTable['id']
PlayerMatches['Team number'] = [bigTable[bigTable['id'] == i]['team'].sum() for i in players.keys()]
PlayerMatches['Team'] = [teams[PlayerMatches.at[i,'Team number']] for i in range(len(players))]
PlayerMatches['Team games'] = np.zeros(len(players))
PlayerMatches['Played'] = np.zeros(len(players))

for j in range(lastGW,0,-1):
    PlayerMatches['Played'] = PlayerMatches['Played'] + ([Gameweeks[(Gameweeks['id']==PlayerMatches.iat[i,0])&\
                                    (Gameweeks['gameweek']==j)]['minutes'].sum() \
                                    for i in range(len(PlayerMatches))] != np.zeros(len(PlayerMatches)))

PlayerMatches['Team games'] = PlayerMatches['Team games'] + [len(Gameweeks[Gameweeks['id']==PlayerMatches.iat[i,0]])\
                                    for i in range(len(PlayerMatches))]
print(8)
#1 Players Threat

PlayerThreat = pd.DataFrame()
PlayerThreat['id'] = bigTable['id']
PlayerThreat['Name'] = bigTable['full_name']
PlayerThreat['Team'] = PlayerMatches['Team']
PlayerThreat['Threat per fixture'] = np.zeros(len(players))
PlayerThreat['Threat per game'] = np.zeros(len(players))

for j in range(lastGW,0,-1):
    PlayerThreat['Threat GW'+str(j)] = [Gameweeks[(Gameweeks['id']==PlayerThreat.iat[i,0])&(Gameweeks['gameweek']==j)] \
                                    ['threat'].sum() \
                                    for i in range(len(players))]
    PlayerThreat['Threat per game'] = PlayerThreat['Threat per game'] + PlayerThreat['Threat GW'+str(j)]

PlayerThreat['Threat per fixture'] = PlayerThreat['Threat per game']/noZ(PlayerMatches['Team games'])
PlayerThreat['Threat per game'] = PlayerThreat['Threat per game']/noZ(PlayerMatches['Played'])
print(9)
#2 Players Creativity

PlayerCreativity = pd.DataFrame()
PlayerCreativity['id'] = bigTable['id']
PlayerCreativity['Name'] = bigTable['full_name']
PlayerCreativity['Team'] = PlayerMatches['Team']
PlayerCreativity['Creativity per fixture'] = np.zeros(len(players))
PlayerCreativity['Creativity per game'] = np.zeros(len(players))

for j in range(lastGW,0,-1):
    PlayerCreativity['Creativity GW'+str(j)] = [Gameweeks[(Gameweeks['id']==PlayerThreat.iat[i,0])&\
                    (Gameweeks['gameweek']==j)]['creativity'].sum() for i in range(len(players))]
    PlayerCreativity['Creativity per game'] = PlayerCreativity['Creativity per game']\
        + PlayerCreativity['Creativity GW'+str(j)]

PlayerCreativity['Creativity per fixture'] = PlayerCreativity['Creativity per game']/noZ(PlayerMatches['Team games'])
PlayerCreativity['Creativity per game'] = PlayerCreativity['Creativity per game']/noZ(PlayerMatches['Played'])
print(10)
#3 Players Threat Adjusted

PlayerThreatAd = pd.DataFrame()
PlayerThreatAd['id'] = bigTable['id']
PlayerThreatAd['Name'] = bigTable['full_name']
PlayerThreatAd['Team number'] = PlayerMatches['Team number']
PlayerThreatAd['Team'] = PlayerMatches['Team']
PlayerThreatAd['Threat per fixture adj'] = np.zeros(len(players))
PlayerThreatAd['Threat per game adj'] = np.zeros(len(players))
for j in range(lastGW,0,-1):
        
    PlayerThreatAd['Threat GW'+str(j) + 'adj'] = [PlayerThreat.at[i,'Threat GW'+str(j)]*threatAllowedAv/ \
            TableDefence.at[toint(Gameweeks[(Gameweeks['team']==PlayerThreatAd.at[i,'Team number'])&\
            (Gameweeks['gameweek']==j)]['teamAgainst'].mean()-1), \
            'Threat allowed av']  for i in range(len(players))]                     
    
    PlayerThreatAd['Threat per game adj'] = PlayerThreatAd['Threat per game adj'] +\
            PlayerThreatAd['Threat GW'+str(j) + 'adj']
PlayerThreatAd['Threat per fixture adj'] = PlayerThreatAd['Threat per game adj']/noZ(PlayerMatches['Team games'])
PlayerThreatAd['Threat per game adj'] = PlayerThreatAd['Threat per game adj']/noZ(PlayerMatches['Played'])
print(11)

#4 PLayers Creativity Adjusted

PlayerCreativityAd = pd.DataFrame()
PlayerCreativityAd['id'] = bigTable['id']
PlayerCreativityAd['Name'] = bigTable['full_name']
PlayerCreativityAd['Team number'] = PlayerMatches['Team number']
PlayerCreativityAd['Team'] = PlayerMatches['Team']
PlayerCreativityAd['Creativity per fixture adj'] = np.zeros(len(players))
PlayerCreativityAd['Creativity per game adj'] = np.zeros(len(players))

for j in range(lastGW,0,-1):
        
    PlayerCreativityAd['Creativity GW'+str(j) + 'adj'] = [PlayerCreativity.at[i,'Creativity GW'+str(j)]*threatAllowedAv/ \
            TableDefence.at[toint(Gameweeks[(Gameweeks['team']==PlayerCreativityAd.at[i,'Team number'])&\
                                          (Gameweeks['gameweek']==j)]['teamAgainst'].mean()-1), \
            'Threat allowed av']  for i in range(len(players))]                     
    
    PlayerCreativityAd['Creativity per game adj'] = PlayerCreativityAd['Creativity per game adj'] +\
            PlayerCreativityAd['Creativity GW'+str(j) + 'adj']

PlayerCreativityAd['Creativity per fixture adj'] = PlayerCreativityAd['Creativity per game adj']/\
            noZ(PlayerMatches['Team games'])
PlayerCreativityAd['Creativity per game adj'] = PlayerCreativityAd['Creativity per game adj']/\
            noZ(PlayerMatches['Played'])
print(12)


#Tables2Files

del TeamThreat['id']
TeamThreat.sort_values('Threat av', ascending = False, inplace = True)
TeamThreat.index = np.arange(1, len(TeamThreat) + 1)
TeamThreat.to_csv(Path('out/TeamThreat.csv'))

del TeamCreativity['id']
TeamCreativity.sort_values('Creativity av', ascending = False, inplace = True)
TeamCreativity.index = np.arange(1, len(TeamCreativity) + 1)
TeamCreativity.to_csv(Path('out/TeamCreativity.csv'))

del TableDefence['id']
TableDefence.sort_values('Threat allowed av', ascending = True, inplace = True)
TableDefence.index = np.arange(1, len(TableDefence) + 1)
TableDefence.to_csv(Path('out/TableDefence.csv'))

del TeamThreatAd['id']
TeamThreatAd.sort_values('Threat av adj', ascending = False, inplace = True)
TeamThreatAd.index = np.arange(1, len(TeamThreatAd) + 1)
TeamThreatAd.to_csv(Path('out/TeamThreatAd.csv'))

del TeamCreativityAd['id']
TeamCreativityAd.sort_values('Creativity av adj', ascending = False, inplace = True)
TeamCreativityAd.index = np.arange(1, len(TeamCreativityAd) + 1)
TeamCreativityAd.to_csv(Path('out/TeamCreativityAd.csv'))

del TableDefenceAd['id']
TableDefenceAd.sort_values('Threat allowed av adj', ascending = True, inplace = True)
TableDefenceAd.index = np.arange(1, len(TableDefenceAd) + 1)
TableDefenceAd.to_csv(Path('out/TableDefenceAd.csv'))

del TableTeams['id']
TableTeams.sort_values('Threat adjusted', ascending = False, inplace = True)
TableTeams.index = np.arange(1, len(TableTeams) + 1)
TableTeams.to_csv(Path('out/TableTeams.csv'))

del PlayerThreat['id']
PlayerThreat.sort_values('Threat per fixture', ascending = False, inplace = True)
PlayerThreat.index = np.arange(1, len(players) + 1)
PlayerThreat.to_csv(Path('out/PlayerThreat.csv'))

del PlayerCreativity['id']
PlayerCreativity.sort_values('Creativity per fixture', ascending = False, inplace = True)
PlayerCreativity.index = np.arange(1, len(players) + 1)
PlayerCreativity.to_csv(Path('out/PlayerCreativity.csv'))

del PlayerThreatAd['id']
del PlayerThreatAd['Team number']
PlayerThreatAd.sort_values('Threat per fixture adj', ascending = False, inplace = True)
PlayerThreatAd.index = np.arange(1, len(players) + 1)
PlayerThreatAd.to_csv(Path('out/PlayerThreatAd.csv'))

del PlayerCreativityAd['id']
del PlayerCreativityAd['Team number']
PlayerCreativityAd.sort_values('Creativity per fixture adj', ascending = False, inplace = True)
PlayerCreativityAd.index = np.arange(1, len(players) + 1)
PlayerCreativityAd.to_csv(Path('out/PlayerCreativityAd.csv'))

PlayerThreatAd

0
1
2
3
4
5
6
7
8
9
10
11
12


,Name,Team,Threat per fixture adj,Threat per game adj,Threat GW14adj,Threat GW13adj,Threat GW12adj,Threat GW11adj,Threat GW10adj,Threat GW9adj,Threat GW8adj,Threat GW7adj,Threat GW6adj,Threat GW5adj,Threat GW4adj,Threat GW3adj,Threat GW2adj,Threat GW1adj
1,Raheem Sterling,Man City,54.169794,58.336702,75.155567,8.946700,93.254636,38.486189,59.162283,32.675779,51.697032,94.108402,0.000000,66.403846,9.641404,56.821519,105.580451,66.443314
2,Roberto Firmino,Liverpool,52.830113,52.830113,34.709055,55.367293,44.308407,32.788735,97.791729,62.049242,60.280474,11.901034,86.484762,23.435607,74.308796,83.290254,26.773001,46.133198
3,Mohamed Salah,Liverpool,50.517826,58.937464,28.924212,0.000000,73.338053,24.947950,80.483459,0.000000,138.204015,80.927031,53.680197,58.993079,59.859863,56.368152,24.263032,27.260526
4,Raúl Jiménez,Wolves,45.978280,45.978280,122.580650,76.053418,29.224742,97.592621,10.505617,19.243094,45.836283,12.390497,24.506834,19.384516,66.866496,44.378864,31.024621,44.107664
5,Teemu Pukki,Norwich,42.319596,42.319596,23.556840,29.718443,47.655757,38.565616,22.563361,22.728608,49.895901,29.952798,45.410931,50.419912,21.549183,56.662431,92.126179,61.668388
6,Sergio Agüero,Man City,39.913363,46.565590,0.000000,52.189081,66.180709,92.868847,0.000000,0.000000,48.531907,8.667879,53.374448,62.209919,55.920144,59.444051,44.136090,15.264004
7,Marcus Rashford,Man Utd,39.768375,39.768375,14.968770,49.984343,88.700917,20.980253,56.618016,36.098569,1.616249,31.969997,5.387296,29.405109,98.725441,26.322155,9.495373,86.484762
8,Jamie Vardy,Leicester,39.653818,39.653818,85.440523,91.593339,40.383153,36.306421,77.809034,21.673399,15.041070,51.719960,38.943609,5.640840,39.337975,32.132792,14.911166,4.220166
9,Tammy Abraham,Chelsea,37.992264,40.914746,0.000000,32.085398,22.691513,72.436751,5.160333,50.911836,29.282970,74.238811,37.602676,60.137364,72.596307,33.551417,7.351277,33.845041
10,Danny Ings,Southampton,37.366696,37.366696,53.374448,21.874208,43.339395,10.695133,1.470255,86.513400,90.958112,38.943609,38.463797,2.380207,33.845041,12.533825,88.742315,0.000000
